In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

import pandas as pd
df = pd.read_csv('crop_data.csv')

In [2]:
!curl -o D:/iot-data/sensor-data.json https://uitiotprocess.s3.amazonaws.com/sensor-data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: Failed to create the file D:/iot-data/sensor-data.json: No such file 
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (23) Failure writing output to destination


In [ ]:
df['Sum'] = df[['N', 'P', 'K']].sum(axis=1)

# Define recommended actions based on rules (example rules)
def get_recommended_action(row):
    if row['N'] < 3:
        return 'Add Fertilizer'
    if row['P'] < 3:
        return 'Add Phosphorus'
    if row['rainfall'] < 150:
        return 'Increase Watering'
    if row['humidity'] < 60:
        return 'Provide More Humidity'
    if row['N'] > 4 and row['P'] > 4 and row['K'] > 4:
        return 'Optimal Nutrient Levels'
    return 'No Specific Action'


# Apply the rules to generate recommended actions
df['Recommended_Action'] = df.apply(get_recommended_action, axis=1)

In [ ]:
df.dropna(inplace=True)
label_encoder = LabelEncoder()
df['Recommended_Action_Encoded'] = label_encoder.fit_transform(df['Recommended_Action'])

X = df[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
y = df['Recommended_Action_Encoded']

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_dist = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}
svm = SVC()
random_search = RandomizedSearchCV(estimator=svm,
                                   param_distributions=param_dist,
                                   n_iter=10,
                                   cv=3,
                                   verbose=1,
                                   n_jobs=-1,
                                   scoring='accuracy',
                                   random_state=42)

random_search.fit(X_train, y_train)
best_svm = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)

y_pred = best_svm.predict(X_test)

y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels = label_encoder.inverse_transform(y_test)

print("SVM Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_, zero_division=1))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Parameters: {'kernel': 'linear', 'gamma': 'scale', 'C': 100}
SVM Classification Report:
                         precision    recall  f1-score   support

         Add Fertilizer       1.00      1.00      1.00        11
      Increase Watering       0.99      1.00      1.00       339
     No Specific Action       0.00      1.00      0.00         0
Optimal Nutrient Levels       1.00      0.97      0.99        80
  Provide More Humidity       1.00      0.80      0.89        10

               accuracy                           0.99       440
              macro avg       0.80      0.96      0.77       440
           weighted avg       0.99      0.99      0.99       440

